In [162]:
import numpy as np
import pandas as pd

pd.set_option("display.width", 1000)

In [163]:
logs = [
    "../log-simple-cloudflare",
    "../log-single-cloudflare",
    "../log-simple-client1",
    "../log-single-client1",
    "../log-multi-client1",
    "../log"
]


In [164]:
def analyze(df: pd.DataFrame) -> float:
    df = df[
        ((df["protocol"] == "h2") | (df["protocol"] == "h3")) & (df["status"] == 200)
    ]
    # print(f"{df.size=}")

    df_groupby = df[["n", "startMili", "endMili", "baseUnixTime"]].groupby("n")

    print(f'start = {df_groupby.min()["baseUnixTime"].min()}')
    print(f'end = {df_groupby.max()["baseUnixTime"].max()}')

    df_groupby_count = df_groupby.count()["startMili"]
    df_success = df_groupby_count == df_groupby_count.mode()[0]
    # print(f"{df_success[df_success].size=}")

    df_mins = df_groupby.min()["startMili"]
    df_maxs = df_groupby.max()["endMili"]
    duration_df = df_maxs - df_mins
    duration_df = duration_df[df_success]

    duration_df_describe = duration_df.describe()

    df_lower_limit = duration_df_describe["mean"] - duration_df_describe["std"] * 3
    df_upper_limit = duration_df_describe["mean"] + duration_df_describe["std"] * 3

    duration_df_filtered = duration_df[
        (df_lower_limit < duration_df) & (duration_df < df_upper_limit)
    ]

    df_des = duration_df_filtered.describe()

    return pd.DataFrame([duration_df_describe, df_des], index=["prefilterd", "filterd"])


In [165]:
def compare(log: str):
    h2 = pd.read_json(log + "/h2-performances-processed.json")
    h3 = pd.read_json(log + "/h3-performances-prosessed.json")
    # print()
    print(log)
    print("h2")
    result_h2 = analyze(h2)
    print(result_h2)
    print()
    print("h3")
    result_h3 = analyze(h3)
    print(result_h3)

    print()


In [166]:
for log in logs:
    compare(log)

../log-simple-cloudflare
h2
start = 1674663191207
end = 1674664568583
             count        mean       std    min    25%    50%    75%    max
prefilterd  1000.0  112.022000  8.741735  100.0  107.0  109.0  113.0  161.0
filterd      992.0  111.752016  8.209357  100.0  107.0  109.0  113.0  138.0

h3
start = 1674665170091
end = 1674666529490
            count       mean       std   min   25%   50%   75%    max
prefilterd  999.0  89.338338  8.616387  75.0  84.0  86.0  89.0  165.0
filterd     992.0  89.031250  7.685154  75.0  84.0  86.0  89.0  115.0

../log-single-cloudflare
h2
start = 1674715385259
end = 1674718244255
            count         mean         std     min     25%     50%     75%     max
prefilterd  807.0  1532.852540  154.885828  1417.0  1463.5  1508.0  1564.0  4769.0
filterd     801.0  1524.285893   85.435762  1417.0  1463.0  1507.0  1562.0  1989.0

h3
start = 1674718847176
end = 1674721704000
            count         mean         std     min     25%     50%     75%     m

In [167]:
import json

h2_open = open("../log-simple-cloudflare" + "/h2-performances.json", "r")
h3_open = open("../log-simple-cloudflare" + "/h3-performances.json", "r")

h2_load = json.load(h2_open)
h3_load = json.load(h3_open)

h2_open.close()
h3_open.close()


In [168]:
from itertools import chain


h2_entrieses = [{"number": x["number"], **x["entries"][0]} for x in h2_load]
h2_entries = entries = pd.DataFrame(chain(h2_entrieses))
h3_entrieses = [{"number": x["number"], **x["entries"][0]} for x in h3_load]
h3_entries = entries = pd.DataFrame(chain(h3_entrieses))

difference = h3_entries.select_dtypes(include=float) / h2_entries.select_dtypes(
    include=float
)
difference = difference.replace([np.inf, -np.inf], np.nan)
difference = difference.dropna()
difference.mean()
# difference.mean()


duration                      0.918611
fetchStart                    1.064796
domainLookupStart             0.997054
domainLookupEnd               0.997686
connectStart                  0.997686
connectEnd                    0.802522
secureConnectionStart         0.701130
requestStart                  0.801311
responseStart                 0.858383
responseEnd                   0.884958
domInteractive                0.918079
domContentLoadedEventStart    0.918100
domContentLoadedEventEnd      0.918789
domComplete                   0.918653
loadEventStart                0.918609
loadEventEnd                  0.918611
dtype: float64